# **Checklist**

o que falta fazer: 

- construir gráficos para a análise desses dados
- ver se precisa tirar algum outlier (dados discrepantes ou inconsistentes) -> *acho que não precisa pq o outlier que tem é justamente no preço e nós não vamos usar essa coluna*
- transformação dos dados usando algum encoder (transformar categóricos em numéricos)
- dividir em treino e teste (em porcentagem, ex: 20 pra teste e 80 pra treino)
- previsão do preço
- regressão linear
- gráficos sobre os ultimos 2
- finalizar com comentários

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import csv
import plotly.express as px

In [2]:
df = pd.read_csv("/content/AB_NYC_2019.csv")

In [3]:
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [4]:
df.sample(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
40082,31119139,"Beautiful, sunlit bedroom in Brooklyn",33839663,Mike,Brooklyn,Greenpoint,40.72663,-73.93819,Private room,70,7,2,2019-05-16,0.32,2,63
35926,28539464,"#2BED Luxury Midtown Full Kitchen, Comp Breakfast",214347105,Roman,Manhattan,Midtown,40.75306,-73.96662,Entire home/apt,399,3,13,2019-06-09,1.48,8,229


In [5]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [6]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


**iremos fazer a previsão do preço a partir dos restantes dos dados**

previsão: price

colunas de base: 
  - neighbourhood_group, neighbourhood, room_type *(categoricos)*
  - minimum_nights, number_of_reviews, reviews_per_month,		calculated_host_listings_count, availability_365 *(numéricos)*


  os dados categóricos teremos que transformar para numéricos
  não sei se todas essas colunas vão ser relevantes..falta analisar isso

# --- Limpeza dos Dados ---

In [7]:
# checagem de valores nulos no df
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [8]:
# retirando os valores nulos para a coluna 'name'

df.dropna(subset = ['name'], inplace = True) # o inplace é pra garantir que o df atualize os dados substituídos
df.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10042
reviews_per_month                 10042
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [9]:
# retirando os valores nulos para a coluna 'host_name'

df.dropna(subset = ['host_name'], inplace = True)
df.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                             0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10037
reviews_per_month                 10037
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [10]:
# análise dos valores nulos para number_of_reviews
df.loc[df['number_of_reviews'] == 0].head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
19,7750,Huge 2 BR Upper East Cental Park,17985,Sing,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190,7,0,NaN,NaN,2,249
26,8700,Magnifique Suite au N de Manhattan - vue Cloitres,26394,Claude & Sophie,Manhattan,Inwood,40.86754,-73.92639,Private room,80,4,0,NaN,NaN,1,0
36,11452,Clean and Quiet in Brooklyn,7355,Vt,Brooklyn,Bedford-Stuyvesant,40.68876,-73.94312,Private room,35,60,0,NaN,NaN,1,365
38,11943,Country space in the city,45445,Harriet,Brooklyn,Flatbush,40.63702,-73.96327,Private room,150,1,0,NaN,NaN,1,365


In [11]:
# substituindo os valores NaN da coluna reviews_per_month para 0

df['reviews_per_month'] = df['reviews_per_month'].fillna(0.0)

In [12]:
# fazendo a checagem apos a limpeza

df.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                             0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10037
reviews_per_month                     0
calculated_host_listings_count        0
availability_365                      0
dtype: int64

# --- Transformação dos Dados (tá errado por enquanto) ---

In [17]:
X_teste = df.iloc[:, [4, 5, 8, 9, 10, 11, 13, 14, 15]].values
X_teste

array([['Brooklyn', 'Kensington', 'Private room', ..., 0.21, 6, 365],
       ['Manhattan', 'Midtown', 'Entire home/apt', ..., 0.38, 2, 355],
       ['Manhattan', 'Harlem', 'Private room', ..., 0.0, 1, 365],
       ...,
       ['Manhattan', 'Harlem', 'Entire home/apt', ..., 0.0, 1, 27],
       ['Manhattan', "Hell's Kitchen", 'Shared room', ..., 0.0, 6, 2],
       ['Manhattan', "Hell's Kitchen", 'Private room', ..., 0.0, 1, 23]],
      dtype=object)

In [18]:
y_teste = df.iloc[:,9].values

In [19]:
from sklearn.preprocessing import LabelEncoder

X_neighbourhood_group = LabelEncoder()
X_neighbourhood = LabelEncoder()
X_room_type = LabelEncoder()

In [20]:
X_teste[:,4] = X_neighbourhood_group.fit_transform(X_teste[:,4])
X_teste[:,5] = X_neighbourhood.fit_transform(X_teste[:,5])
X_teste[:,8] = X_room_type.fit_transform(X_teste[:,8])

X_teste

array([['Brooklyn', 'Kensington', 'Private room', ..., 0.21, 6, 365],
       ['Manhattan', 'Midtown', 'Entire home/apt', ..., 0.38, 2, 355],
       ['Manhattan', 'Harlem', 'Private room', ..., 0.0, 1, 365],
       ...,
       ['Manhattan', 'Harlem', 'Entire home/apt', ..., 0.0, 1, 27],
       ['Manhattan', "Hell's Kitchen", 'Shared room', ..., 0.0, 6, 2],
       ['Manhattan', "Hell's Kitchen", 'Private room', ..., 0.0, 1, 23]],
      dtype=object)